# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4439026712#4310029237#4088494919
date='December 6, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,11.8km/h,53.3km/h
1,Calories,1292,1292
2,Elapsed Time,2:25:01,2:25:01


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,A subir al colegio El Salado 0.11km 5m -3.9%,40s,9.9km/h,297W,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
1,"CASETA COCACOLA, PUENTE ARENALES 1.40km 119m...",12:40,6.7km/h,212W,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
2,La Catedral (hasta la U) 3.45km 410m 11.9%,35:16,5.9km/h,185W,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
3,Catedral Km0-0.5 - 8.3% 0.50km 42m 8.3%,3:41,8.2km/h,211W,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
4,La Catedral (completa) 6.45km 656m 10.0%,1:00:51,6.4km/h,187W,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
...,...,...,...,...,...
90,NaN,NaN,8:55,6.7km/h,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
91,NaN,NaN,8:47,6.9km/h,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
92,NaN,NaN,8:07,7.1km/h,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."
93,NaN,NaN,5:29,7.9km/h,"{'Calories': {'Avg': '1292', 'Max': '1292'}, '..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df['istr']=df['Info'].astype(str)

In [9]:
df=df.drop_duplicates(['Name', 'Power', 'Speed', 'Time', 'istr']).drop('istr',axis='columns').reset_index(drop=True)

In [10]:
df.to_json(db_file)

### Save notebook here

In [11]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [12]:
!bash commit.sh

[master 5ec05e8] Morning Ride 5:44 AM on Tuesday, January 26, 2021
 2 files changed, 268 insertions(+), 513 deletions(-)
 rewrite Strava.ipynb (61%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 3.42 KiB | 218.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   fe70e65..5ec05e8  master -> master


In [13]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.96)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Linux 4.19.0-10-amd64 x86_64)
